<a href="https://colab.research.google.com/github/talgiladi/medical-chatbot/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install langchain-pinecone

In [118]:
from load_dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")


In [42]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers

In [3]:
PINECONE_API_ENV=""

HOST="https://medical-chatbot-fn63rwe.svc.aped-4627-b74a.pinecone.io"
INDEX="quickstart"

In [5]:
def load_pdf(directory: str):
    loader = DirectoryLoader(directory,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs

In [10]:
docs = load_pdf(directory="data/")

In [12]:
def text_split(documents: list):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 50
    )
    chunks = splitter.split_documents(documents=documents)
    return chunks

In [105]:
text_chunks = text_split(docs)

In [23]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name = "sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

In [91]:
embeddings = download_hugging_face_embeddings()

In [ ]:
from langchain_pinecone import PineconeVectorStore
docsearch = None
index_name = "medical-chatbot2"
has_existing_index = True
if (has_existing_index):
    docsearch = PineconeVectorStore.from_existing_index(
        index_name = index_name, 
        embedding=embeddings)
else:
    #upload the data now
    batch_size = 300  # Adjust this value based on your requirement
    chunks = [t.page_content for t in text_chunks]
    
    # Split the chunks array into smaller arrays
    chunks_small = [chunks[i:i + batch_size] for i in range(0, len(chunks), batch_size)]
  
    # Loop through each small array and upload it
    for i, small_chunk in enumerate(chunks_small):
        print(f"Uploading batch {i+1} out of {len(chunks_small)}")
        docsearch=PineconeVectorStore.from_texts(small_chunk, embeddings, index_name=index_name)
        print(f"Finished uploading batch {i+1}")

In [150]:
#%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain
from langchain_pinecone import PineconeVectorStore

index_name = "medical-chatbot2"

docsearch = PineconeVectorStore.from_existing_index(embedding= embeddings, index_name=index_name)


In [153]:
 test1 = docsearch.similarity_search("allergies")
 for i, d in enumerate(test1):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

thecondition known as allergy, and the offending substanceis called an allergen.

## Document 1

condition known as allergy, and the offend-ing substance is called an allergen. Common inhaledallergens include pollen, dust, and insect parts from tinyhouse mites. Common food allergens include nuts, fish,and milk.

## Document 2

GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE

## Document 3

the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .
The particular allergens to which a person is sensi-


In [137]:
from langchain.prompts import PromptTemplate


prompt_template = """you are a medical chat bot. If you don't know the answer, say that you don't know, don't try to make things up.
    context:{context}
    question:{question}
    """
prompt=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": prompt}

In [134]:
from langchain.llms import CTransformers
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [160]:
from langchain.chains import LLMChain
query = "tell my a few words about allergies"
retriever = docsearch.as_retriever()
matched_docs = docsearch.similarity_search(query=  query, k = 3)
context = ""
for i, d in enumerate(test1):
    context = context + (f"\n## Document {i}\n")
    context = context + (d.page_content)

chain = LLMChain(llm = llm, prompt = prompt)

In [162]:
result = chain.run(question= query, context= context, verbose = True)

In [164]:
print(result)

 answer:Allergy is an abnormal reaction to substances that are normally harmless to most people. these substances, known as allergen, can trigger a wide range of symptoms including sneezing, runny nose, congestion, itchy eyes and throat, coughing, wheezing, and skin rashes. Common allergens include pollen, dust, mold, pet dander, insect stings, and certain foods such as nuts, fish, and milk.
